In [295]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np

''' NOTES
    - A player will ALWAYS have an ADP, but not necessarily a bye (not on a team). This means that WebElement that is
    assigned to the "elem_bye_adps" variable doesn't always follow the "bye" then "adp" order
    - ASSUMING that every player has a position - this is currently the case, but might not be in the future 
    (doubt it though)
    - Headless option doesn't seem to return anything for the main data elements
'''

adp_url = "https://underdogfantasy.com/rankings/NFL/87a5caba-d5d7-46d9-a798-018d7c116213"
# url = "http://www.python.org"

CHROMEDRIVER_PATH = r"C:\Users\conde\chromedriver\chromedriver.exe"

CLASS_NAME_ALL_PLAYERS = 'styles__playerCellWrapper__1cf1R'

CLASS_NAME_PLAYER_NAME = 'styles__playerName__1EFeQ'
CLASS_NAME_POSITION = 'styles__playerPosition__2KPHG'
CLASS_NAME_BYE_ADPS = 'styles__statValue__3BAOt'
CLASS_NAME_BYE_ADP_IDS = 'styles__statKey__1xe-s'

options = Options()
options.add_argument('--headless')

# driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)
driver = webdriver.Chrome(CHROMEDRIVER_PATH)
driver.get(adp_url)

In [296]:
elem = driver.find_elements_by_class_name('styles__field__3fmc7')[0]
elem.clear()
elem.send_keys("condelong11@yahoo.com")
elem.send_keys(Keys.RETURN)

elem = driver.find_elements_by_class_name('styles__field__3fmc7')[1]
elem.clear()
elem.send_keys("PenciL1!")
elem.send_keys(Keys.RETURN)

In [346]:
def process_player_data(elem_player):
    ''' Pulls out the Player name, Position, Bye, and ADP for a web element representing an individual player '''
    
    player_name = elem_player.find_elements_by_class_name(CLASS_NAME_PLAYER_NAME)[0].text
    player_pos = elem_player.find_elements_by_class_name(CLASS_NAME_POSITION)[0].text
    
    elem_bye_adps = elem_player.find_elements_by_class_name(class_name_bye_adps)
    elem_bye_adp_ids = elem_player.find_elements_by_class_name(class_name_bye_adp_ids)
    
    # Set the variables for instances where these aren't available
    player_bye = 0
    player_adp = 999
    for bye_adp_index in range(len(elem_bye_adp_ids)):
        if elem_bye_adp_ids[bye_adp_index].text == 'Bye':
            player_bye = elem_bye_adps[bye_adp_index].text
            
        elif elem_bye_adp_ids[bye_adp_index].text == 'ADP':
            player_adp = elem_bye_adps[bye_adp_index].text

    player_data = [player_name, player_pos, player_bye, player_adp]
    
    return player_data


def process_all_players_data():
    
    elem_all_players = driver.find_elements_by_class_name(CLASS_NAME_ALL_PLAYERS)
    
    all_player_data = []
    for rank, elem_player in enumerate(elem_all_players):
        player_data = process_player_data(elem_player)
        
        # Add a rank variable to the front of the list
        rank_list = [rank]
        rank_list.extend(player_data)
        
        all_player_data.append(rank_list)
        
    cols = ['Rank', 'Player', 'Position/Team', 'Bye', 'ADP']
    final_df = pd.DataFrame(all_player_data, columns=cols)
    
    # Replacing ADP with Rank when the ADP is missing
    final_df['ADP'] = np.where(final_df['ADP'] == '-', final_df['Rank'], final_df['ADP'])
    
    # Create index to slice the Position/Team column
    final_df['pos_team_slice_index'] = final_df['Position/Team'].str.find(' - ')
    final_df.loc[final_df['pos_team_slice_index'] == -1, 'pos_team_slice_index'] = final_df['Position/Team'].str.len()

    # Split out the 'Position/Team' column
    final_df['Position'] = final_df.apply(lambda x: x['Position/Team'][0:x['pos_team_slice_index']], 1)
    final_df.loc[final_df['Position/Team'].str.len() != final_df['pos_team_slice_index'], 'Team'] = final_df.apply(
        lambda x: x['Position/Team'][x['pos_team_slice_index'] + 3:], 1)
    
    final_cols = ['Rank', 'Player', 'Position', 'Team', 'Bye', 'ADP']
    final_df = final_df[final_cols]
    
    return final_df


adp_df = process_all_players_data()
print(adp_df)

      Rank               Player Position Team Bye   ADP
0        0  Christian McCaffrey       RB  CAR  13   1.0
1        1          Dalvin Cook       RB  MIN   7   2.1
2        2        Derrick Henry       RB  TEN  13   3.8
3        3         Alvin Kamara       RB   NO   6   4.4
4        4      Ezekiel Elliott       RB  DAL   7   5.9
...    ...                  ...      ...  ...  ..   ...
1114  1114          Kevin White       WR   SF   6  1114
1115  1115         Keyion Dixon       WR  NaN   0  1115
1116  1116          Myles Fells       RB  NaN   0  1116
1117  1117        Austin Proehl       WR   SF   6  1117
1118  1118         Chandler Cox       RB  NaN   0  1118

[1119 rows x 6 columns]


In [344]:
import numpy as np
# print(adp_df.dtypes)
# check = adp_df.groupby('ADP').count()
# print(check)

check = adp_df.copy()

# Create index to slice the Position/Team column
check['pos_team_slice_index'] = check['Position/Team'].str.find(' - ')
check.loc[check['pos_team_slice_index'] == -1, 'pos_team_slice_index'] = check['Position/Team'].str.len()

# Split out the 'Position/Team' column
check['Position'] = check.apply(lambda x: x['Position/Team'][0:x['pos_team_slice_index']], 1)
check.loc[check['Position/Team'].str.len() != check['pos_team_slice_index'], 'Team'] = check.apply(
    lambda x: x['Position/Team'][x['pos_team_slice_index'] + 3:], 1)

check = check['Position']
check



# check = check.groupby('pos_team_slice_index').count()
# check

0       RB
1       RB
2       RB
3       RB
4       RB
        ..
1114    WR
1115    WR
1116    RB
1117    WR
1118    RB
Name: Position, Length: 1119, dtype: object

In [294]:
import pandas as pd

cols = ['Player', 'Position/Team', 'Bye', 'ADP']

check = pd.DataFrame(test, columns=cols)

check

,Player,Position/Team,Bye,ADP
0,Christian McCaffrey,RB - CAR,13,1.0
1,Dalvin Cook,RB - MIN,7,2.1
2,Derrick Henry,RB - TEN,13,3.8
3,Alvin Kamara,RB - NO,6,4.3
4,Ezekiel Elliott,RB - DAL,7,5.9
...,...,...,...,...
1114,Nathan Peterman,QB - LV,8,-
1115,Jay Cutler,QB,0,-
1116,Luke Stocker,TE - TEN,13,-
1117,Jake Burt,TE,0,-


In [293]:
rank = 1
test_list = [1, 2, 3]

new_rank = [rank]
test_list = new_rank.extend(test_list)

print(test_list)

None


# SCRATCH

In [277]:

# elem_names = driver.find_elements_by_class_name(class_name_name)
# elem_positions = driver.find_elements_by_class_name(class_name_position)

# elem_bye_adps = driver.find_elements_by_class_name(class_name_bye_adps)
# elem_bye_adp_ids = driver.find_elements_by_class_name(class_name_bye_adp_ids)

def read_elements(driver_obj, class_name):
    ''' Reads the main attributes of every element and stores it in a dictionary '''
    
    elems = driver_obj.find_elements_by_class_name(class_name)
    
    text_list = []
    location_list = []
    for elem in elems:
        text_list.append(elem.text)
        location_list.append(elem.location)
        
    final_dict = {'text': text_list, 'location': location_list}
        
    return final_dict

elem_names = read_elements(driver, class_name_player_name)
# elem_positions = read_elements(driver, class_name_position)
# elem_bye_adps = read_elements(driver, class_name_bye_adps)
# elem_bye_adp_ids = read_elements(driver, class_name_bye_adp_ids)

print(len(elem_names))

2


In [286]:
# all_players = driver.find_elements_by_class_name('styles__playerCellWrapper__1cf1R')

check = all_players[0].find_elements_by_class_name(CLASS_NAME_BYE_ADP_IDS)

print(len(check))
print(check[0].text)

2
Bye


In [276]:
def process_bye_adp_data(web_elem_bye_adps, web_elem_bye_adp_ids, num_players):
    ''' Process the adp data such that it aligns with the player name and position data'''
    
    final_bye_adps = []
    bye_adp_id_index = 0
    for player_index in range(num_players):

        check = bye_adp_id_index
        
        try:
            check = web_elem_bye_adp_ids[bye_adp_id_index]
        except:
            print(player_index, bye_adp_id_index)
        
        if web_elem_bye_adp_ids[bye_adp_id_index] == 'Bye':
            bye = web_elem_bye_adps[bye_adp_id_index]
            adp = web_elem_bye_adps[bye_adp_id_index + 1]
                
            bye_adp_id_index += 2
        elif web_elem_bye_adp_ids[bye_adp_id_index] == 'ADP':
            bye = 0
            adp = web_elem_bye_adps[bye_adp_id_index]
            
            bye_adp_id_index += 1
        else:
            # Set the variables in case neither data points are available
            bye = 0
            adp = 999
            
            bye_adp_id_index += 2
            
        final_bye_adps.append([bye, adp, bye_adp_id_index])
            
    return final_bye_adps


check = process_bye_adp_data(elem_bye_adps, elem_bye_adp_ids, len(elem_names))

1012 2024


IndexError: list index out of range

In [262]:
print(len(elem_names))

1119


In [263]:
def process_bye_adp_data_v2(web_elem_bye_adps, web_elem_bye_adp_ids, num_players):
    ''' Process the adp data such that it aligns with the player name and position data'''

    final_bye_adps = []
    bye_adp_id_index = 0
    for player_index in range(num_players):

        check = bye_adp_id_index
        
        if web_elem_bye_adp_ids[bye_adp_id_index].text == 'Bye':
            bye = web_elem_bye_adps[bye_adp_id_index].text
            adp = web_elem_bye_adps[bye_adp_id_index + 1].text
                
            bye_adp_id_index += 2
        elif web_elem_bye_adp_ids[bye_adp_id_index].text == 'ADP':
            bye = 0
            adp = web_elem_bye_adps[bye_adp_id_index + 1].text
            
            bye_adp_id_index += 1
        else:
            # Set the variables in case neither data points are available
            bye = 0
            adp = 999
            
            bye_adp_id_index += 2
            
        final_bye_adps.append([bye, adp, bye_adp_id_index])
            
    return final_bye_adps


check = process_bye_adp_data(elem_bye_adps, elem_bye_adp_ids, len(elem_names))

529


In [265]:
print(check[-4:])

[[0, '6'], ['6', '-'], ['14', '-'], ['9', '-']]


In [221]:
def process_bye_adp_data_old(web_elem_bye_adps, web_elem_bye_adp_ids, num_players):
    ''' Process the adp data such that it aligns with the player name and position data'''
    
    bye_id_loc = web_elem_bye_adps[0].location['x']
    adp_id_loc = web_elem_bye_adps[1].location['x']
    
    print(web_elem_bye_adp_ids[0].location['x'])
    
    final_bye_adps = []
    bye_adp_id_index = 0
    for player_index in range(num_players):
         # Set the variables in case neither data points are available
        bye = 0
        adp = 999
        
        print(web_elem_bye_adp_ids[bye_adp_id_index].location['x'], " ", bye_adp_id_index)
        
        if web_elem_bye_adp_ids[bye_adp_id_index].location['x'] == bye_id_loc:            
            if web_elem_bye_adp_ids[bye_adp_id_index].text == 'Bye':
                bye = web_elem_bye_adps[bye_adp_id_index].text
                adp = web_elem_bye_adps[bye_adp_id_index + 1].text
                
            bye_adp_id_index += 2
        else:
            if web_elem_bye_adp_ids[bye_adp_id_index].text == 'ADP':
                bye = 0
                adp = web_elem_bye_adps[bye_adp_id_index].text

            bye_adp_id_index += 1
            
        final_bye_adps.append([bye, adp])
        
        if player_index > 5:
            break
            
    return final_bye_adps


check = process_bye_adp_data(elem_bye_adps, elem_bye_adp_ids, len(elem_names))

529
529   0
566   1
529   2
566   3
529   4
566   5
529   6


In [226]:
for check in elem_bye_adps:
#     if check.location['x'] not in(529, 566):
#         print(check.location['x'])
    print(check.location)

{'x': 533, 'y': 210}
{'x': 570, 'y': 210}
{'x': 537, 'y': 271}
{'x': 570, 'y': 271}
{'x': 533, 'y': 331}
{'x': 570, 'y': 331}
{'x': 537, 'y': 391}
{'x': 570, 'y': 391}
{'x': 537, 'y': 451}
{'x': 570, 'y': 451}
{'x': 533, 'y': 512}
{'x': 570, 'y': 512}
{'x': 533, 'y': 572}
{'x': 570, 'y': 572}
{'x': 533, 'y': 632}
{'x': 570, 'y': 632}
{'x': 537, 'y': 692}
{'x': 570, 'y': 692}
{'x': 532, 'y': 752}
{'x': 565, 'y': 752}
{'x': 537, 'y': 813}
{'x': 565, 'y': 813}
{'x': 532, 'y': 873}
{'x': 565, 'y': 873}
{'x': 532, 'y': 933}
{'x': 565, 'y': 933}
{'x': 537, 'y': 993}
{'x': 565, 'y': 993}
{'x': 532, 'y': 1054}
{'x': 565, 'y': 1054}
{'x': 532, 'y': 1114}
{'x': 565, 'y': 1114}
{'x': 537, 'y': 1174}
{'x': 565, 'y': 1174}
{'x': 532, 'y': 1234}
{'x': 565, 'y': 1234}
{'x': 537, 'y': 1294}
{'x': 565, 'y': 1294}
{'x': 532, 'y': 1355}
{'x': 565, 'y': 1355}
{'x': 537, 'y': 1415}
{'x': 565, 'y': 1415}
{'x': 537, 'y': 1475}
{'x': 565, 'y': 1475}
{'x': 532, 'y': 1535}
{'x': 565, 'y': 1535}
{'x': 537, 'y': 

{'x': 556, 'y': 11736}
{'x': 523, 'y': 11796}
{'x': 556, 'y': 11796}
{'x': 528, 'y': 11857}
{'x': 556, 'y': 11857}
{'x': 523, 'y': 11917}
{'x': 556, 'y': 11917}
{'x': 523, 'y': 11977}
{'x': 556, 'y': 11977}
{'x': 523, 'y': 12037}
{'x': 556, 'y': 12037}
{'x': 523, 'y': 12098}
{'x': 556, 'y': 12098}
{'x': 528, 'y': 12158}
{'x': 556, 'y': 12158}
{'x': 523, 'y': 12218}
{'x': 556, 'y': 12218}
{'x': 528, 'y': 12278}
{'x': 556, 'y': 12278}
{'x': 523, 'y': 12338}
{'x': 556, 'y': 12338}
{'x': 528, 'y': 12399}
{'x': 556, 'y': 12399}
{'x': 523, 'y': 12459}
{'x': 556, 'y': 12459}
{'x': 523, 'y': 12519}
{'x': 556, 'y': 12519}
{'x': 523, 'y': 12579}
{'x': 556, 'y': 12579}
{'x': 523, 'y': 12640}
{'x': 556, 'y': 12640}
{'x': 528, 'y': 12700}
{'x': 556, 'y': 12700}
{'x': 528, 'y': 12760}
{'x': 556, 'y': 12760}
{'x': 528, 'y': 12820}
{'x': 556, 'y': 12820}
{'x': 528, 'y': 12880}
{'x': 556, 'y': 12880}
{'x': 523, 'y': 12941}
{'x': 556, 'y': 12941}
{'x': 523, 'y': 13001}
{'x': 556, 'y': 13001}
{'x': 528, 

{'x': 556, 'y': 22840}
{'x': 528, 'y': 22901}
{'x': 556, 'y': 22901}
{'x': 523, 'y': 22961}
{'x': 556, 'y': 22961}
{'x': 523, 'y': 23021}
{'x': 556, 'y': 23021}
{'x': 523, 'y': 23081}
{'x': 556, 'y': 23081}
{'x': 523, 'y': 23142}
{'x': 556, 'y': 23142}
{'x': 523, 'y': 23202}
{'x': 556, 'y': 23202}
{'x': 528, 'y': 23262}
{'x': 556, 'y': 23262}
{'x': 528, 'y': 23322}
{'x': 556, 'y': 23322}
{'x': 528, 'y': 23382}
{'x': 556, 'y': 23382}
{'x': 528, 'y': 23443}
{'x': 556, 'y': 23443}
{'x': 528, 'y': 23503}
{'x': 556, 'y': 23503}
{'x': 523, 'y': 23563}
{'x': 556, 'y': 23563}
{'x': 523, 'y': 23623}
{'x': 556, 'y': 23623}
{'x': 523, 'y': 23684}
{'x': 556, 'y': 23684}
{'x': 528, 'y': 23744}
{'x': 556, 'y': 23744}
{'x': 528, 'y': 23804}
{'x': 556, 'y': 23804}
{'x': 528, 'y': 23864}
{'x': 556, 'y': 23864}
{'x': 528, 'y': 23924}
{'x': 556, 'y': 23924}
{'x': 528, 'y': 23985}
{'x': 556, 'y': 23985}
{'x': 506, 'y': 24045}
{'x': 557, 'y': 24045}
{'x': 494, 'y': 24105}
{'x': 545, 'y': 24105}
{'x': 528, 

{'x': 533, 'y': 34643}
{'x': 579, 'y': 34643}
{'x': 579, 'y': 34704}
{'x': 533, 'y': 34764}
{'x': 579, 'y': 34764}
{'x': 579, 'y': 34824}
{'x': 579, 'y': 34884}
{'x': 537, 'y': 34944}
{'x': 579, 'y': 34944}
{'x': 579, 'y': 35005}
{'x': 533, 'y': 35065}
{'x': 579, 'y': 35065}
{'x': 533, 'y': 35125}
{'x': 579, 'y': 35125}
{'x': 533, 'y': 35185}
{'x': 579, 'y': 35185}
{'x': 579, 'y': 35246}
{'x': 579, 'y': 35306}
{'x': 537, 'y': 35366}
{'x': 579, 'y': 35366}
{'x': 533, 'y': 35426}
{'x': 579, 'y': 35426}
{'x': 579, 'y': 35486}
{'x': 533, 'y': 35547}
{'x': 579, 'y': 35547}
{'x': 533, 'y': 35607}
{'x': 579, 'y': 35607}
{'x': 537, 'y': 35667}
{'x': 579, 'y': 35667}
{'x': 579, 'y': 35727}
{'x': 579, 'y': 35788}
{'x': 533, 'y': 35848}
{'x': 579, 'y': 35848}
{'x': 579, 'y': 35908}
{'x': 579, 'y': 35968}
{'x': 537, 'y': 36028}
{'x': 579, 'y': 36028}
{'x': 579, 'y': 36089}
{'x': 579, 'y': 36149}
{'x': 537, 'y': 36209}
{'x': 579, 'y': 36209}
{'x': 579, 'y': 36269}
{'x': 533, 'y': 36330}
{'x': 579, 

{'x': 579, 'y': 48132}
{'x': 579, 'y': 48193}
{'x': 537, 'y': 48253}
{'x': 579, 'y': 48253}
{'x': 579, 'y': 48313}
{'x': 537, 'y': 48373}
{'x': 579, 'y': 48373}
{'x': 579, 'y': 48433}
{'x': 533, 'y': 48494}
{'x': 579, 'y': 48494}
{'x': 533, 'y': 48554}
{'x': 579, 'y': 48554}
{'x': 537, 'y': 48614}
{'x': 579, 'y': 48614}
{'x': 579, 'y': 48674}
{'x': 579, 'y': 48735}
{'x': 537, 'y': 48795}
{'x': 579, 'y': 48795}
{'x': 533, 'y': 48855}
{'x': 579, 'y': 48855}
{'x': 533, 'y': 48915}
{'x': 579, 'y': 48915}
{'x': 537, 'y': 48975}
{'x': 579, 'y': 48975}
{'x': 537, 'y': 49036}
{'x': 579, 'y': 49036}
{'x': 579, 'y': 49096}
{'x': 537, 'y': 49156}
{'x': 579, 'y': 49156}
{'x': 579, 'y': 49216}
{'x': 537, 'y': 49277}
{'x': 579, 'y': 49277}
{'x': 579, 'y': 49337}
{'x': 579, 'y': 49397}
{'x': 579, 'y': 49457}
{'x': 537, 'y': 49517}
{'x': 579, 'y': 49517}
{'x': 579, 'y': 49578}
{'x': 537, 'y': 49638}
{'x': 579, 'y': 49638}
{'x': 533, 'y': 49698}
{'x': 579, 'y': 49698}
{'x': 537, 'y': 49758}
{'x': 579, 

{'x': 533, 'y': 61621}
{'x': 579, 'y': 61621}
{'x': 533, 'y': 61682}
{'x': 579, 'y': 61682}
{'x': 537, 'y': 61742}
{'x': 579, 'y': 61742}
{'x': 537, 'y': 61802}
{'x': 579, 'y': 61802}
{'x': 533, 'y': 61862}
{'x': 579, 'y': 61862}
{'x': 579, 'y': 61922}
{'x': 537, 'y': 61983}
{'x': 579, 'y': 61983}
{'x': 537, 'y': 62043}
{'x': 579, 'y': 62043}
{'x': 533, 'y': 62103}
{'x': 579, 'y': 62103}
{'x': 579, 'y': 62163}
{'x': 537, 'y': 62224}
{'x': 579, 'y': 62224}
{'x': 537, 'y': 62284}
{'x': 579, 'y': 62284}
{'x': 533, 'y': 62344}
{'x': 579, 'y': 62344}
{'x': 579, 'y': 62404}
{'x': 579, 'y': 62464}
{'x': 579, 'y': 62525}
{'x': 533, 'y': 62585}
{'x': 579, 'y': 62585}
{'x': 579, 'y': 62645}
{'x': 533, 'y': 62705}
{'x': 579, 'y': 62705}
{'x': 533, 'y': 62766}
{'x': 579, 'y': 62766}
{'x': 533, 'y': 62826}
{'x': 579, 'y': 62826}
{'x': 537, 'y': 62886}
{'x': 579, 'y': 62886}
{'x': 533, 'y': 62946}
{'x': 579, 'y': 62946}
{'x': 579, 'y': 63006}
{'x': 579, 'y': 63067}
{'x': 579, 'y': 63127}
{'x': 533, 

In [157]:
all_player_data = []
elem_bye_adp_index = 0

check_all = []
for i, player in enumerate(elem_names):
    if i != 1112:
        if elem_bye_adp_ids[elem_bye_adp_index].text == 'Bye':
            elem_bye_adp_index += 2
        else:
            elem_bye_adp_index += 1
            
        check = [player.text, elem_bye_adp_index]
        
        check_all.append(check)
        continue
    else:
        player_data = []

        player_name = player.text
        player_position = elem_positions[i].text

        # Accounting for not every player having a bye week and therefore the bye/adp pattern in the elem_bye_adp_ids
        # list is not consistent
        print(elem_bye_adp_index)
        print(elem_bye_adp_ids[elem_bye_adp_index].text)
        if elem_bye_adp_ids[elem_bye_adp_index].text == 'Bye':
            player_bye = elem_bye_adps[elem_bye_adp_index].text
            player_adp = elem_bye_adps[elem_bye_adp_index + 1].text

            elem_bye_adp_index += 2
        else:
            player_bye = 'N/A'
            player_adp = elem_bye_adps[elem_bye_adp_index].text

            elem_bye_adp_index += 1
        
        player_data.append([player_name, player_position, player_bye, player_adp])

        all_player_data.append(player_data)
        
print(all_player_data)

1959
Bye
[[['Michael Roberts', 'TE', '8', '-']]]


In [144]:
print(check_all[0:5])

[['Christian McCaffrey', 2], ['Dalvin Cook', 4], ['Derrick Henry', 6], ['Alvin Kamara', 8], ['Ezekiel Elliott', 10]]


In [159]:
for i, player in enumerate(check_all):
    if player[1] % 2 != 0:
        print(player, " ", i)
        break
        
print(check_all[225])

['Anthony Miller', 455]   227
['Josh Reynolds', 452]


In [205]:
# print(len(elem_bye_adp_ids))
print(elem_bye_adp_ids[1].text)
print(elem_bye_adp_ids[0].location['x'])
print(elem_bye_adp_ids[1].location)

print(elem_bye_adp_ids[-11].location)
print(elem_bye_adp_ids[-12].location)

ADP
529
{'x': 566, 'y': 234}
{'x': 566, 'y': 67306}
{'x': 566, 'y': 67246}


In [123]:
print(len(elem_names))
print(elem_names[1113].text)

1119
Aca'Cedric Ware


In [99]:
print(elem_positions[1].text)

print(elem_bye_adp_ids[-11].text, ": ", elem_bye_adps[-1].text)
print(elem_bye_adp_ids[-12].text, ": ", elem_bye_adps[-2].text)

RB - MIN
ADP :  -
ADP :  12


In [100]:
print(elem_bye_adp_id[0].text)
print(elem_bye_adp_id[1].text)

Bye
ADP


In [61]:
html1 = driver.page_source

html2 = driver.execute_script("return document.documentElement.innerHTML;")

In [62]:
soup = BeautifulSoup(html2, 'html.parser')

In [63]:
print(soup.head)

<head><meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-control"/><script async="" src="https://connect.facebook.net/signals/config/643926619550303?v=2.9.44&amp;r=stable"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-RPPC68JN0L&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-5KVTG9J"></script><script src="https://www.gstatic.com/firebasejs/7.19.0/firebase-app.js"></script><script src="https://www.gstatic.com/firebasejs/7.19.0/firebase-analytics.js"></script><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
      new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
      j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
      'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
      })(window,d